# DESI Fisher examples

In this notebook we will run some DESI forecasts.

In [1]:
import numpy as np

zedges = np.linspace(1.1, 1.6, 6)
zranges = list(zip(zedges[:-1], zedges[1:]))
nbars = [1079.9998, 1036.0009, 970.9995, 876.9994, 553.9985]  # nobj / deg^2 / dz
b0 = 0.84  # at z = 0

In [2]:
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import BAOPowerSpectrumTemplate, SimpleBAOWigglesTracerPowerSpectrumMultipoles, DampedBAOWigglesTracerPowerSpectrumMultipoles
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, CutskyFootprint, ObservablesCovarianceMatrix
from desilike.likelihoods.galaxy_clustering import ObservablesGaussianLikelihood, SNWeightedPowerSpectrumLikelihood
from desilike import Fisher, setup_logging

#setup_logging()
cosmo = DESI()
fo = cosmo.get_fourier()

covariances = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))

    fisher = Fisher(likelihood)
    precision = fisher(**params)
    #covariance = precision.select(params=['qiso']).to_covariance()
    covariance = precision.to_covariance(params=['qiso'])
    #print(covariance.to_stats(tablefmt='pretty'))
    covariances.append(covariance)

/home/adematti/anaconda3/envs/cosmodesi/lib/python3.9/site-packages/cosmoprimo-1.0.0-py3.9.egg/cosmoprimo/eisenstein_hu.py:23: UserWarning: EisensteinHuNoWiggleEngine cannot cope with massive neutrinos
  warnings.warn('{} cannot cope with massive neutrinos'.format(self.__class__.__name__))


In [3]:
# Let's compare against reference
refs = [0.7019, 0.7107, 0.7296, 0.7658, 1.0062]
print(100. * np.array([cov.std(params='qiso') for cov in covariances]) / np.array(refs)[:len(covariances)])

[1.03700001 1.04829564 1.06127027 1.07709428 1.11326464]


In [4]:
# Let's compare with multipole compression
covariances2 = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    observable = TracerPowerSpectrumMultipolesObservable(data=params,  # data can be a dictionary of parameters
                                                     klim={0: [0.01, 0.5], 2: [0.01, 0.5], 4: [0.01, 0.5]}, # fit monopole and quadrupole, between 0.02 and 0.2 h/Mpc
                                                     kstep=0.005, theory=theory)
    covariance = ObservablesCovarianceMatrix(observable, footprints=footprint, resolution=5)
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(**params))
    fisher = Fisher(likelihood)
    precision = fisher(**params)
    #covariance = precision.select(params=['qiso']).to_covariance()
    covariance = precision.to_covariance(params=['qiso'])
    covariances2.append(covariance)

/home/adematti/anaconda3/envs/cosmodesi/lib/python3.9/site-packages/numpy/lib/polynomial.py:1329: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [5]:
print(np.array([cov.std(params='qiso') for cov in covariances2]) / np.array([cov.std(params='qiso') for cov in covariances]))

[1.00082489 1.00074818 1.00066009 1.00055065 1.00015226]
